# ML 101

## Time Series

In this project, we will use simple model named TSM that models a time series as a markov chain of level one to model the behaviour of the time series and predict the next values.

In [ ]:
!pip install paho-mqtt plotly

In [ ]:
import numpy as np

class TSM:
  def __init__(self, w, h, min=0, max=100):
    self.w = w
    self.h = h
    self.min = min
    self.max = max
    self.ts = 0
    self.total = 0
    self.previous_value = None
    self.probs = np.zeros((w, h, h))
  
  def width(self):
    return self.w
  
  def height(self):
    return self.h
  
  def average_value(self, k):
    inc = (self.max - self.min) / self.h
    return self.min + (k*inc) + (inc/2.0)
  
  def value_bin(self, value):
    inc = (self.max - self.min) / self.h
    idx = 0
    i = self.min

    while i < (value-inc):
      i += inc
      idx += 1

    return idx
  
  def fit(self, value):
    # update the min and max values
    if self.max is None or self.max < value:
      self.max = value
    
    if self.min is None or self.min > value:
      self.min = value
    
    # compute the right bin
    idx = self.value_bin(value)
    if self.previous_value is not None:
      self.probs[self.ts][self.previous_value][idx] += 1
      self.ts = (self.ts + 1) % self.w
      self.total += 1
    
    self.previous_value = idx
  
  def predict(self):
    p = self.probs[self.ts][self.previous_value]
    k = np.argmax(p)
    return self.average_value(k)
  
  def model(self):
    inc = (self.max - self.min) / self.h
    X = []
    for i in range(0, self.w):
      for j in range(0, self.h):
        p = self.probs[i][j]
        X.append([i, self.min+(j*inc), np.amax(p)])
    return np.array(X)

In [ ]:
import paho.mqtt.client as mqtt


tTSM = TSM(10,20, 0, 40)
hTSM = TSM(10,20, 0, 100)
pTSM = TSM(10,20, 100000, 103000)


def on_connect(client, userdata, flags, rc):
    client.subscribe('unave/tem')
    client.subscribe('unave/hum')
    client.subscribe('unave/pre')


def on_message(client, userdata, msg):
    input = float(msg.payload.decode("utf-8"))
    if msg.topic == 'unave/tem':
      tTSM.fit(input)
      value = tTSM.predict()
      client.publish("predict/tem", value)
    elif msg.topic == 'unave/hum':
      hTSM.fit(input)
      value = hTSM.predict()
      client.publish("predict/hum", value)
    else:
      pTSM.fit(input)
      value = pTSM.predict()
      client.publish("predict/pre", value)


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message


client.connect("192.168.0.100", 1883, 60)
client.loop_start()

In [ ]:
client.disconnect() #disconnect
client.loop_stop() #stop loop

In [ ]:
import plotly.express as px
import numpy as np

X = hTSM.model()
fig = px.scatter_3d(x=X[:,0], y=X[:, 1], z=X[:, 2])
fig.show(renderer="colab")